In [10]:
import sys
import os
sys.path.append("..")
from models import CustomResNet, DINOv2ViT
import torch
from dinov2.layers.dino_head import DINOHead
from torch import nn
import yaml
from models import get_teacher_and_student
with open("config/config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

teacher, student = get_teacher_and_student(cfg)
teacher.to(device)
student.to(device)


/storage/disk0/arda/dinov2/distillation/../dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/storage/disk0/arda/dinov2/distillation/../dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/storage/disk0/arda/dinov2/distillation/../dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")
Using cache found in /home/arda/.cache/torch/hub/facebookresearch_dinov2_main


ModuleDict(
  (backbone): CustomResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(


: 

In [2]:
img = torch.randn(1, 3, 140, 280).to(device)
student["backbone"].eval()
teacher["backbone"].eval()

student_out = student["backbone"](img)
teacher_out = teacher["backbone"](img)

print(student_out["patch_embeddings"].shape)
print(student["ibot_head"](student_out["patch_embeddings"]).shape)
print(teacher["ibot_head"](teacher_out["patch_embeddings"]).shape)
print(student["dino_head"](student_out["embedding"]).shape)
print(teacher["dino_head"](teacher_out["embedding"]).shape)
# print(student["dino_head"]().shape)



torch.Size([1, 200, 2048])
torch.Size([1, 200, 2048])
torch.Size([1, 200, 2048])
torch.Size([1, 2048])
torch.Size([1, 2048])


/home/arda/miniconda3/envs/dinov2/lib/python3.9/site-packages/xformers/ops/unbind.py:46: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage_data_ptr = tensors[0].storage().data_ptr()
/home/arda/miniconda3/envs/dinov2/lib/python3.9/site-packages/xformers/ops/unbind.py:48: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if x.storage().data_ptr() != storage_data_ptr:


In [3]:
from datasets.GTA5 import GTA5Dataset
from datasets.collate_fn import collate_data_and_cast
from dinov2.data.augmentations import DataAugmentationDINO
from torch.utils.data import DataLoader


import yaml

# Load configurations
with open("config/config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

# Data Transformation
data_transform = DataAugmentationDINO(
    global_crops_scale=tuple(cfg['data_transform']['global_crops_scale']),
    local_crops_scale=tuple(cfg['data_transform']['local_crops_scale']),
    local_crops_number=cfg['data_transform']['n_local_crops'],
    global_crops_size=tuple(cfg['data_transform']['global_crops_size']),
    local_crops_size=tuple(cfg['data_transform']['local_crops_size']),
)

# Dataset and DataLoader
dataset = GTA5Dataset(transform=data_transform)
# data_loader = DataLoader(
#     dataset,
#     batch_size=cfg['data_loader']['batch_size'],
#     num_workers=cfg['data_loader']['num_workers'],
#     shuffle=cfg['data_loader']['shuffle'],
#     collate_fn=collate_data_and_cast
# )

# Split dataset into train and test sets
from torch.utils.data import random_split

# Calculate lengths for 80-20 split
total_size = len(dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

# Create train and test datasets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create train and test dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=cfg['data_loader']['batch_size'], 
    num_workers=cfg['data_loader']['num_workers'],
    shuffle=cfg['data_loader']['shuffle'],
    collate_fn=collate_data_and_cast
)

test_loader = DataLoader(
    test_dataset,
    batch_size=cfg['data_loader']['batch_size'],
    num_workers=cfg['data_loader']['num_workers'], 
    shuffle=False,
    collate_fn=collate_data_and_cast
)

# Freeze all teacher head parameters
for param in teacher["dino_head"].parameters():
    param.requires_grad = False
    
for param in teacher["ibot_head"].parameters():
    param.requires_grad = False

# Verify teacher heads are frozen
for name, param in teacher["dino_head"].named_parameters():
    assert not param.requires_grad, f"Parameter {name} in dino_head is not frozen"
    
for name, param in teacher["ibot_head"].named_parameters():
    assert not param.requires_grad, f"Parameter {name} in ibot_head is not frozen"


# Optimizer
optimizer = getattr(torch.optim, cfg['optimizer']['type'])([
    {"params": student.parameters()},
], lr=cfg['optimizer']['lr'])


> In the original architecture, they are using same head for dino and ibot loss maybe we can try it

In [4]:

from dinov2.loss.dino_clstoken_loss import DINOLoss
from dinov2.loss.ibot_patch_loss import iBOTPatchLoss
from tqdm import tqdm

dino_loss_weight = cfg["dino_loss"]["weight"]
ibot_loss_weight = cfg["ibot_loss"]["weight"]
loss_scales = 2
cosine_similarity = nn.CosineSimilarity(dim=1)
dino_loss = DINOLoss(out_dim=cfg["teacher"]["dino_head"]["out_dim"], student_temp=cfg["dino_loss"]["student_temp"])
ibot_patch_loss = iBOTPatchLoss(patch_out_dim=cfg["teacher"]["ibot_head"]["out_dim"], student_temp=cfg["ibot_loss"]["student_temp"])

# Freeze all teacher backbone parameters
for param in teacher["backbone"].parameters():
    param.requires_grad = False

# Verify parameters are frozen
for name, param in teacher["backbone"].named_parameters():
    assert not param.requires_grad, f"Parameter {name} is not frozen"

for epoch in range(cfg["train"]["max_epochs"]):
    epoch_loss = []
    student.train()
    teacher.eval()
    for i, data in enumerate(tqdm(train_loader)):
        cosine_similarities = []
        loss_dict = {}
        loss_accumulator = 0
        current_batch_size = data["collated_global_crops"].shape[0] / 2
        n_global_crops = cfg["data_transform"]["n_global_crops"]
        n_local_crops = cfg["data_transform"]["n_local_crops"]
        global_crops = data["collated_global_crops"]
        local_crops = data["collated_local_crops"]

        global_crops = global_crops.to(device)
        local_crops = local_crops.to(device)

        n_local_crops_loss_terms = max(n_local_crops * n_global_crops, 1)
        n_global_crops_loss_terms = (n_global_crops - 1) * n_global_crops
        ibot_loss_scale = 1.0 / n_global_crops
        with torch.no_grad():
            teacher_backbone_output_dict = teacher["backbone"](global_crops)
            teacher_cls_tokens = teacher_backbone_output_dict["embedding"]
            teacher_cls_tokens = teacher_cls_tokens.chunk(n_global_crops)
            # watch out: these are chunked and cat'd in reverse so A is matched to B in the global crops dino loss
            teacher_cls_tokens = torch.cat((teacher_cls_tokens[0], teacher_cls_tokens[1]))

            ibot_teacher_patch_tokens = teacher_backbone_output_dict["patch_embeddings"]
            _dim = ibot_teacher_patch_tokens.shape[-1]
            n_cls_tokens = teacher_cls_tokens.shape[0]

    

        # TODO: Try to use same head for both
        teacher_cls_tokens_after_head = teacher.dino_head(teacher_cls_tokens) # (B*2, D)
        teacher_patch_tokens_after_head = teacher.ibot_head(ibot_teacher_patch_tokens) #(B*2, #patches, D) 

        teacher_dino_softmaxed = torch.softmax(teacher_cls_tokens_after_head / cfg["dino_loss"]["teacher_temp"], dim=-1)
        teacher_ibot_softmaxed = torch.softmax(teacher_patch_tokens_after_head / cfg["ibot_loss"]["teacher_temp"], dim=-1)
        
        student_global_backbone_output_dict = student.backbone(global_crops)
        student_local_backbone_output_dict = student.backbone(local_crops)

        inputs_for_student_head_list = []
        student_local_cls_tokens = student_local_backbone_output_dict["embedding"]
        # inputs_for_student_head_list.append(student_local_cls_tokens.unsqueeze(0)) # (1, B, D)

        student_global_cls_tokens = student_global_backbone_output_dict["embedding"]
        # inputs_for_student_head_list.append(student_global_cls_tokens.unsqueeze(0)) # (1, B, D)

                
        ibot_student_patch_tokens = student_global_backbone_output_dict["patch_embeddings"]
        student_global_patch_tokens_after_head = student.ibot_head(ibot_student_patch_tokens)

        student_local_cls_tokens_after_head = student.dino_head(student_local_cls_tokens)    
        student_global_cls_tokens_after_head = student.dino_head(student_global_cls_tokens)

        if n_local_crops > 0:
            dino_local_crops_loss = dino_loss(
                student_output_list=student_local_cls_tokens_after_head.chunk(n_local_crops),
                teacher_out_softmaxed_centered_list=teacher_dino_softmaxed.chunk(2),
            ) 

            dino_local_crops_loss /= (n_global_crops_loss_terms + n_local_crops_loss_terms)

            # store for display
            loss_dict["dino_local_crops_loss"] = dino_local_crops_loss

            # accumulate loss
            loss_accumulator += dino_loss_weight * dino_local_crops_loss

        dino_global_crops_loss = (
                        dino_loss(
                        student_output_list=student_global_cls_tokens_after_head.chunk(n_global_crops),
                        teacher_out_softmaxed_centered_list=teacher_dino_softmaxed.chunk(2),
                    )
                    * loss_scales
                    / (n_global_crops_loss_terms + n_local_crops_loss_terms)
                )
        
        loss_dict["dino_global_crops_loss"] = dino_global_crops_loss

                # accumulate loss
        loss_accumulator += dino_loss_weight * dino_global_crops_loss
        

        
        # compute loss
        ibot_patch_l = (
                ibot_patch_loss.forward(
                student_global_patch_tokens_after_head,
                teacher_ibot_softmaxed,
            )
            * loss_scales
            * ibot_loss_scale
        )

        # store for display
        loss_dict["ibot_loss"] = ibot_patch_l / 2

        # accumulate loss
        loss_accumulator += ibot_loss_weight * ibot_patch_l

        loss_accumulator.backward()
        optimizer.step()
        optimizer.zero_grad()

        # print(loss_accumulator.item())
        epoch_loss.append(loss_accumulator.item())
        cosine_similarities.append(cosine_similarity(teacher_cls_tokens_after_head, student_global_cls_tokens_after_head).mean().item())

    student.eval()
    cosine_similarities_test = []
    for i, data in enumerate(tqdm(test_loader)):
        
        global_crops = data["collated_global_crops"]
        global_crops = global_crops.to(device)
        local_crops = data["collated_local_crops"]
        local_crops = local_crops.to(device)
        teacher_out = teacher["backbone"](global_crops)
        teacher_cls_tokens = teacher_out["embedding"]
        teacher_cls_tokens_after_head = teacher.dino_head(teacher_cls_tokens)
        student_out = student["backbone"](global_crops)
        student_cls_tokens = student_out["embedding"]
        student_cls_tokens_after_head = student.dino_head(student_cls_tokens)
        cosine_similarities_test.append(cosine_similarity(teacher_cls_tokens_after_head, student_cls_tokens_after_head).mean().item())


    print(f"Epoch {epoch}, Loss: {sum(epoch_loss)/len(epoch_loss)}")
    print(f"Cosine Similarity: {sum(cosine_similarities)/len(cosine_similarities)}")
    print(f"Cosine Similarity Test: {sum(cosine_similarities_test)/len(cosine_similarities_test)}")



100%|██████████| 32/32 [00:31<00:00,  1.02it/s]


Epoch 0, Loss: 17.22791651916504
Cosine Similarity: 0.8483266830444336
Cosine Similarity Test: 0.8534210473299026


100%|██████████| 32/32 [00:30<00:00,  1.04it/s]


Epoch 1, Loss: 17.166044738769532
Cosine Similarity: 0.8610295653343201
Cosine Similarity Test: 0.8640767615288496


100%|██████████| 32/32 [00:31<00:00,  1.01it/s]


...

Epoch 5, Loss: 17.135942596435548
Cosine Similarity: 0.8802197575569153
Cosine Similarity Test: 0.8606315553188324


100%|██████████| 32/32 [00:31<00:00,  1.03it/s]


Epoch 6, Loss: 17.13220457458496
Cosine Similarity: 0.8476824164390564
Cosine Similarity Test: 0.8592491634190083


  4%|▍         | 5/125 [00:12<05:07,  2.56s/it]


KeyboardInterrupt: 

In [ ]:
teacher["backbone"].


In [5]:
# Save only the backbone weights of student model
torch.save({
    'backbone_state_dict': student.backbone.state_dict(),
}, 'student_backbone_checkpoint.pth')

print("Student backbone weights saved successfully")



Student backbone weights saved successfully


In [9]:
# Load the checkpoint
checkpoint = torch.load('/home/arda/dinov2/dinov2/checkpoints/dinov2_vitg14_linear_head.pth')

checkpoint['weight'].shape
checkpoint['bias'].shape



torch.Size([1000])